In [4]:
%pip install wikipedia

import wikipedia
wikipedia.set_lang("de")  # auf Deutsch umstellen

# Beispiel: Artikel laden
page = wikipedia.page("Mathematik")
print(page.title)
print(page.content[:1000])  # zeige ersten 1000 Zeichen


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\tobia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Mathematik
Die Mathematik (bundesdeutsches Hochdeutsch: [matemaˈtiːk], [matemaˈtik]; österreichisches Hochdeutsch: [mateˈmaːtik]; von altgriechisch μαθηματικὴ τέχνη mathēmatikē téchnē ‚die Kunst des Lernens‘) ist eine Formalwissenschaft, die aus der Untersuchung von geometrischen Figuren und dem Rechnen mit Zahlen entstand. Für Mathematik gibt es keine allgemein anerkannte Definition; heute wird sie üblicherweise als eine Wissenschaft beschrieben, die durch logische Definitionen selbstgeschaffene abstrakte Strukturen mittels der Logik auf ihre Eigenschaften und Muster untersucht.


== Geschichte ==

Die Mathematik ist eine der ältesten Wissenschaften. Ihre erste Blüte erlebte sie noch vor der Antike in Mesopotamien, Indien und China, später in der Antike in Griechenland und im Hellenismus. Von dort datiert die Orientierung an der Aufgabenstellung des „rein logischen Beweisens“ und die erste Axiomatisierung, nämlich die e

In [4]:
import requests
import json
import wikipedia
import re

# Sprache auf Deutsch setzen
wikipedia.set_lang("de")

OLLAMA_MODEL = "qwen3:14b"
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_ollama(prompt, model=OLLAMA_MODEL):
    response = requests.post(OLLAMA_URL, json={
        "model": model,
        "prompt": prompt,
        "stream": False
    })
    return response.json()["response"].strip()

def get_article_content(title):
    try:
        return wikipedia.page(title).content
    except Exception as e:
        print(f" Fehler bei Artikel '{title}': {e}")
        return None

def split_into_chunks(text, max_chars=1000):
    paragraphs = text.split("\n")
    chunks, current = [], ""
    for p in paragraphs:
        if len(current) + len(p) < max_chars:
            current += "\n" + p
        else:
            chunks.append(current.strip())
            current = p
    if current:
        chunks.append(current.strip())
    return chunks

def generate_qa_from_text(text_chunk):
    # Prompt 1: Frage generieren mit <think>-Block
    frage_prompt = f"""
Du bist ein Lernassistent für Schüler. Lies den folgenden Text und überlege laut, worüber man eine einfache, sinnvolle Frage stellen könnte. Gib dann eine passende Frage in deutscher Sprache aus.

TEXT:
{text_chunk}

Gib nur die frage aus, ohne weitere Erklärungen.
"""
    frage_result = query_ollama(frage_prompt).strip()

    # Frage = alles nach dem </think>-Tag
    if "</think>" in frage_result:
        frage = frage_result.split("</think>", 1)[1].strip()
    else:
        print("⚠️ Kein </think>-Block gefunden:\n", frage_result)
        return None, None

    # Prompt 2: Antwort mit Denkprozess generieren
    antwort_prompt = f"""
Du bist ein Lernassistent für Schüler. Beantworte die folgende Frage auf Grundlage des bereitgestellten Textes. Denke zuerst laut über die Antwort nach und formuliere dann eine klare Antwort in deutscher Sprache. Achte darauf, dass die Antwort vollständig und verständlich ist.

TEXT:
{text_chunk}

FRAGE:
{frage}


"""
    antwort = query_ollama(antwort_prompt)

    return frage, antwort


def build_dataset_from_category(category_term, max_articles=100, max_chunks_per_article=5, output_file=None):
    if output_file is None:
        output_file = f"output_{category_term}_dataset.jsonl"
    print(f" Suche nach Artikeln zum Thema: {category_term}")
    search_results = wikipedia.search(category_term, results=max_articles)
    print(f"🔗 Gefundene Artikel: {len(search_results)} , {search_results}")

    dataset = []

    for i, title in enumerate(search_results):
        print(f"\n ({i+1}/{len(search_results)}) Artikel: {title}")
        content = get_article_content(title)
        if not content:
            continue

        chunks = split_into_chunks(content)
        for j, chunk in enumerate(chunks[:max_chunks_per_article]):
            print(f"   ➤ Generiere Q&A ({j+1}/{max_chunks_per_article})...")
            frage, antwort = generate_qa_from_text(chunk)
            if frage and antwort:
                dataset.append({
    "conversations": [
        {"role": "user", "content": frage},
        {"role": "assistant", "content": antwort}
    ]
})


    with open(output_file, "w", encoding="utf-8") as f:
        for item in dataset:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"\n Gesamt: {len(dataset)} Einträge gespeichert in {output_file}")

# Beispielnutzung
if __name__ == "__main__":
    fach = "Algebra, Gleichungen, Funktionen"  # z. B. Biologie, Geschichte, Chemie
    build_dataset_from_category(fach, max_articles=100, max_chunks_per_article=5)


 Suche nach Artikeln zum Thema: Algebra, Gleichungen, Funktionen
🔗 Gefundene Artikel: 100 , ['Cardanische Formeln', 'Quadratische Gleichung', 'Satz von Vieta', 'Algebraische Funktion', 'Eigenwerte und Eigenvektoren', 'Lineare Gleichung', 'Boolesche Algebra', 'Lösen von Gleichungen', 'Binomische Formeln', 'Funktion (Mathematik)', 'Lineare Algebra', 'Matrix (Mathematik)', 'Lineare Funktion', 'Laplace-Operator', 'Analytische Geometrie', 'Geradengleichung', 'Polynom vierten Grades', 'Analysis', 'Quadratische Ergänzung', 'Taschenrechner', 'Polynom', 'Gleichung', 'Computeralgebrasystem', 'Kubische Gleichung', 'Algebraische Struktur', 'Algebra über einem Körper', 'Universelle Algebra', 'Komplexe Zahl', 'Körper (Algebra)', 'Teilgebiete der Mathematik', 'Variable (Mathematik)', 'Komposition (Mathematik)', 'Lambertsche W-Funktion', 'Funktionalanalysis', 'Zahlentheorie', 'Dirac-Gleichung', 'Surjektive Funktion', 'Ganzrationale Funktion', 'Clifford-Algebra', 'Integralgleichung', 'Al-Chwarizmi', 'P

C:\Users\tobia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\tobia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


 Fehler bei Artikel 'Idempotenz': "impotenz" may refer to: 
Erektile Dysfunktion
Anejakulation
Impotentia generandi
Impotentia

 (58/100) Artikel: Satz von Abel-Ruffini
   ➤ Generiere Q&A (1/5)...
   ➤ Generiere Q&A (2/5)...
   ➤ Generiere Q&A (3/5)...
   ➤ Generiere Q&A (4/5)...
   ➤ Generiere Q&A (5/5)...

 (59/100) Artikel: Faltung (Mathematik)
   ➤ Generiere Q&A (1/5)...
   ➤ Generiere Q&A (2/5)...
   ➤ Generiere Q&A (3/5)...
   ➤ Generiere Q&A (4/5)...
   ➤ Generiere Q&A (5/5)...

 (60/100) Artikel: Gewöhnliche Differentialgleichung
   ➤ Generiere Q&A (1/5)...
   ➤ Generiere Q&A (2/5)...
   ➤ Generiere Q&A (3/5)...
   ➤ Generiere Q&A (4/5)...
   ➤ Generiere Q&A (5/5)...

 (61/100) Artikel: Holomorphe Funktion
   ➤ Generiere Q&A (1/5)...
   ➤ Generiere Q&A (2/5)...
   ➤ Generiere Q&A (3/5)...
   ➤ Generiere Q&A (4/5)...
   ➤ Generiere Q&A (5/5)...

 (62/100) Artikel: Lineare Unabhängigkeit
 Fehler bei Artikel 'Lineare Unabhängigkeit': Page id "lineare abhängigkeit" does not match a

In [3]:
import wikipedia

# Sprache auf Deutsch setzen
wikipedia.set_lang("de")

def zeige_wikipedia_artikel(fachbegriff, max_treffer=100):
    print(f"🔎 Suche Wikipedia-Artikel zu: {fachbegriff}")
    artikel = wikipedia.search(fachbegriff, results=max_treffer)
    
    if not artikel:
        print("❌ Keine Artikel gefunden.")
    else:
        print(f"✅ {len(artikel)} Artikel gefunden:")
        for i, titel in enumerate(artikel, 1):
            print(f"{i}. {titel}")

# Beispiel
zeige_wikipedia_artikel("Algebra, Gleichungen, Funktionen")

🔎 Suche Wikipedia-Artikel zu: Algebra, Gleichungen, Funktionen
✅ 100 Artikel gefunden:
1. Cardanische Formeln
2. Quadratische Gleichung
3. Satz von Vieta
4. Eigenwerte und Eigenvektoren
5. Algebraische Funktion
6. Binomische Formeln
7. Lineare Gleichung
8. Lösen von Gleichungen
9. Boolesche Algebra
10. Funktion (Mathematik)
11. Matrix (Mathematik)
12. Lineare Funktion
13. Lineare Algebra
14. Analysis
15. Algebraische Struktur
16. Laplace-Operator
17. Analytische Geometrie
18. Taschenrechner
19. Geradengleichung
20. Quadratische Ergänzung
21. Polynom
22. Polynom vierten Grades
23. Universelle Algebra
24. Kubische Gleichung
25. Gleichung
26. Algebra über einem Körper
27. Computeralgebrasystem
28. Körper (Algebra)
29. Komplexe Zahl
30. Komposition (Mathematik)
31. Funktionalanalysis
32. Zahlentheorie
33. Variable (Mathematik)
34. Lambertsche W-Funktion
35. Teilgebiete der Mathematik
36. Surjektive Funktion
37. Dirac-Gleichung
38. Ganzrationale Funktion
39. Poisson-Klammer
40. Al-Chwarizmi

Note: you may need to restart the kernel to use updated packages.


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-687e82cf-080190a25815525863ed0e03;116c3664-8c18-4f29-9d23-677d78e18d1d)

Repository Not Found for url: https://huggingface.co/api/datasets/Tobronaut/Mathewiki/preupload/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Invalid username or password.
Note: Creating a commit assumes that the repo already exists on the Huggingface Hub. Please use `create_repo` if it's not the case.